# Notebook to test the Carmack algorithm for the inverse problem

In [21]:
import numpy as np
from scipy.special import jn, jn_zeros, jv
from math import factorial as fact
import matplotlib.pyplot as plt

In [ ]:
#load data_clean.npz
data = np.load('data/data_clean.npz')
print(f'Keys in data: {data.files}')

In [23]:
IDX = 10
M = 2
L = 7
FINESSE = 110

Bessel coeffs are 21, 3x7, because there are 2 coefss for cosine (0 and 1), both non-zero, and 1 for sine (1), bc the sine(0) is zero.

In [ ]:
sxr = data['sxr'][IDX]
emiss = data['emiss'][IDX]
bessel_coefss = data['bessel_coefss'][IDX]
a0cl, a1cl, a1sl = bessel_coefss[0:L], bessel_coefss[L:2*L], bessel_coefss[2*L:3*L]

print(f'sxr.shape: {sxr.shape}')
print(f'emiss.shape: {emiss.shape}')
print(f'bessel_coefss.shape: {bessel_coefss.shape}')
print(f'a0cl.shape: {a0cl.shape}')
print(f'a1cl.shape: {a1cl.shape}')
print(f'a1sl.shape: {a1sl.shape}')

## Bessel polynomials

In [ ]:
#bessel
def bes(m:int,x:np.ndarray):
    return jv(m,x)

def bes(m:int,x:np.ndarray,n=20): # accurate until x ~ 15
    b = np.zeros_like(r)
    for l in range(n):
        b += (-1)**l * (x/2)**(2*l+m) / (fact(l)*fact(l+m))
    return b

r = np.linspace(0, 20, 300)
plt.figure(figsize=(12,3))
plt.subplot(121)
for m in range(5):
    plt.plot(r, bes(m,r), label=f'm={m}')
plt.ylim(-1,1)
plt.grid()
plt.legend()
plt.title('Bessel functions by hand')

# plot the first bessel functions
plt.subplot(122)
for m in range(5):
    plt.plot(r, jn(m,r), label=f'm={m}')
plt.ylim(-1,1)
plt.grid()
plt.legend()
plt.title('Bessel functions')